In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from utils.dataloader import DataLoader
from utils.workload import Workload
from utils.cleaner import salvar_csv, salvar_parquet
import sys

# --- Ajuste de path caso necessário ---
sys.path.append("/app")

# --- Inicialização do Spark ---
spark = (
    SparkSession.builder
    .appName("ComparacaoCarregamentoProcessamentoEscrita")
    .master("spark://spark-master:7077")
    .getOrCreate()
)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/10 05:31:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# --- Caminhos ---
caminho_csv_pessoas     = "/app/full_data/pessoas"
caminho_parquet_pessoas = "/app/full_data/parquet_pessoas"
caminho_saida_parquet   = "/app/full_data/output_faixa_etaria_parquet"
caminho_saida_csv       = "/app/full_data/output_faixa_etaria_csv"

# Schema das pessoas (exemplo parcial — adicione o restante conforme necessário)
schema_pessoas = StructType([
    StructField("id", StringType(), True),
    StructField("pesid", StringType(), True),
    StructField("data_inversa", StringType(), True),
    StructField("dia_semana", StringType(), True),
    StructField("horario", StringType(), True),
    StructField("uf", StringType(), True),
    StructField("br", StringType(), True),
    StructField("km", StringType(), True),
    StructField("municipio", StringType(), True),
    StructField("causa_acidente", StringType(), True),
    StructField("tipo_acidente", StringType(), True),
    StructField("classificacao_acidente", StringType(), True),
    StructField("fase_dia", StringType(), True),
    StructField("sentido_via", StringType(), True),
    StructField("condicao_metereologica", StringType(), True),
    StructField("tipo_pista", StringType(), True),
    StructField("tracado_via", StringType(), True),
    StructField("uso_solo", StringType(), True),
    StructField("id_veiculo", StringType(), True),
    StructField("tipo_veiculo", StringType(), True),
    StructField("marca", StringType(), True),
    StructField("ano_fabricacao_veiculo", StringType(), True),
    StructField("tipo_envolvido", StringType(), True),
    StructField("estado_fisico", StringType(), True),
    StructField("idade", StringType(), True),
    StructField("sexo", StringType(), True),
    StructField("nacionalidade", StringType(), True),
    StructField("naturalidade", StringType(), True)
])

print("[ETAPA] Gerando base Parquet a partir do CSV original (se necessário)...")
df_original_csv = DataLoader(spark=spark, base_dir=caminho_csv_pessoas, schema=schema_pessoas).load()
salvar_parquet(df=df_original_csv, output_path=caminho_parquet_pessoas)
print("[ETAPA] Base Parquet gerada.")

[ETAPA] Gerando base Parquet a partir do CSV original (se necessário)...
utf-8


25/07/10 05:31:56 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
[Stage 0:>                                                          (0 + 1) / 1]

[PARQUET] salvo em: /app/full_data/parquet_pessoas
[ETAPA] Base Parquet gerada.


In [3]:
# --- Funções de carregamento ---
def load_csv(spark, path):
    return DataLoader(spark=spark, base_dir=path, schema=schema_pessoas).load()
    
def load_parquet(spark, path):
    return spark.read.parquet(path)

def load_and_materialize(load_fn, spark, path):
    """
    Executa uma função de carregamento e força a materialização do DataFrame em cache.
    Isso garante que o tempo medido inclua a leitura real dos dados do disco.
    """
    df = load_fn(spark, path)
    df.cache()
    df.count()
    return df


def analise_faixa_etaria(df):
    df_limpo = df.withColumn(
        "idade_numerica",
        when(col("idade").rlike("^[0-9]+$"), col("idade").cast(IntegerType()))
        .otherwise(None)
    )

    df_faixa_etaria = df_limpo.withColumn(
        "faixa_etaria",
        when(col("idade_numerica") < 18, "<18")
        .when((col("idade_numerica") >= 18) & (col("idade_numerica") < 30), "18-29")
        .when((col("idade_numerica") >= 30) & (col("idade_numerica") < 45), "30-44")
        .when((col("idade_numerica") >= 45) & (col("idade_numerica") < 60), "45-59")
        .when(col("idade_numerica") >= 60, "60+")
        .otherwise("Desconhecida")
    )

    resultado = df_faixa_etaria.groupBy("faixa_etaria").count().orderBy("faixa_etaria")
    return resultado

# === WORKLOADS ===

# --- WORKLOAD 1: Carregamento Parquet PRIMEIRO ---
df_parquet = Workload.run(
    title="1B - Carregamento e cache de dados (Parquet)",
    execute_fn=lambda spark, path: load_and_materialize(load_parquet, spark, path),
    spark=spark,
    path=caminho_parquet_pessoas
)

# --- WORKLOAD 2: Carregamento CSV DEPOIS ---
df_csv = Workload.run(
    title="1A - Carregamento e cache de dados (CSV)",
    execute_fn=lambda spark, path: load_and_materialize(load_csv, spark, path),
    spark=spark,
    path=caminho_csv_pessoas
)

# --- WORKLOAD 3: Processamento Parquet ---
df_parquet_processado = Workload.run(
    title="2B - Processamento faixa etária (a partir do cache Parquet)",
    execute_fn=analise_faixa_etaria,
    df=df_parquet
)

# --- WORKLOAD 4: Processamento CSV ---
df_csv_processado = Workload.run(
    title="2A - Processamento faixa etária (a partir do cache CSV)",
    execute_fn=analise_faixa_etaria,
    df=df_csv
)

# --- Relatório final com tempos ---
Workload.print_report()

# --- Boas práticas: Liberar memória ---
print("\n[ETAPA] Limpando DataFrames do cache...")
df_csv.unpersist()
df_parquet.unpersist()
print("[ETAPA] Cache limpo.")

# --- Finalização do Spark ---
spark.stop()


[WORKLOAD] 1B - Carregamento e cache de dados (Parquet)


[METRIC] 1B - Carregamento e cache de dados (Parquet) executada em 59.971 segundos
[WORKLOAD] 1A - Carregamento e cache de dados (CSV)
utf-8


[METRIC] 1A - Carregamento e cache de dados (CSV) executada em 42.156 segundos
[WORKLOAD] 2B - Processamento faixa etária (a partir do cache Parquet)
[METRIC] 2B - Processamento faixa etária (a partir do cache Parquet) executada em 0.244 segundos
[WORKLOAD] 2A - Processamento faixa etária (a partir do cache CSV)
[METRIC] 2A - Processamento faixa etária (a partir do cache CSV) executada em 0.341 segundos

[WORKLOAD REPORT]
- [WORKLOAD-1] 1B - Carregamento e cache de dados (Parquet): 59.971s
- [WORKLOAD-2] 1A - Carregamento e cache de dados (CSV): 42.156s
- [WORKLOAD-3] 2B - Processamento faixa etária (a partir do cache Parquet): 0.244s
- [WORKLOAD-4] 2A - Processamento faixa etária (a partir do cache CSV): 0.341s

[ETAPA] Limpando DataFrames do cache...
[ETAPA] Cache limpo.
